In [1]:
import os 
import pandas as pd
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
import h5py

from sklearn.model_selection import KFold #?
from sklearn import preprocessing #?
from sklearn import metrics #?
from scipy import ndimage #?


#import functions_read_data as rdat
# Tensorflow/Keras
import tensorflow as tf
import time
print(tf.__version__)
from tensorflow import keras
from tensorflow.keras import layers
import pickle as pkl

from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

from tensorflow.keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

2.2.0


Using TensorFlow backend.


In [2]:
DIR = "/tf/notebooks/schnemau/xAI_stroke_3d/"
os.chdir(DIR)
DATA_DIR = DIR + "data/"
OUTPUT_DIR = DIR + "weights/10Fold_CIBLSX/"

In [3]:
#ontram functions
from k_ontram_functions.ontram import ontram
from k_ontram_functions.ontram_loss import ontram_loss
from k_ontram_functions.ontram_metrics import ontram_acc, ontram_auc
from k_ontram_functions.ontram_predict import predict_ontram, get_parameters
#data augmentation techniques
from functions.augmentation3d import zoom, rotate, flip, shift
from functions.plot_slices import plot_slices
import functions_read_data as rdat
#my own functions
import Utils_maurice as utils

In [4]:
id_tab = pd.read_csv(DATA_DIR + "10Fold_ids_V0.csv", sep=",") 
X = np.load(DATA_DIR + "prepocessed_dicom_3d.npy")
model_version = 1 # define the model version
print(id_tab.shape)
print(X.shape)

(407, 14)
(407, 128, 128, 28)


# Training

In [5]:
train = True 

# Define Model
layer_connection = "globalAveragePooling" 
last_activation = "linear" 

In [6]:
num_splits = 10   # 10 for all models
num_models = 5    # see overview of all models (above)

batch_size = 6    # 6 for all models
epochs = 250      # 250 for all models

In [7]:
# function for preprocessing
def train_preprocessing(data, label):
    volume = data[0]
    tabular = data[1]
    volume = zoom(volume)
    volume = rotate(volume)
    volume = shift(volume)
    volume = flip(volume)
    return (volume, tabular), label

# Let's get real about it

In [ ]:
# loop over splits (kFold)

start0 = time.time()
for i in range(num_splits):
    start1 = time.time()
    print("\n\n\n\n################################################################################")
    print("Split " + str(i))
    print("################################################################################\n\n\n\n")
    
    (X_train, X_valid, X_test),(X_tab_train, X_tab_valid, X_tab_test), (Y_train, Y_valid, Y_test) = rdat.split_data_tabular(id_tab, X, i)

    X_train = np.expand_dims(X_train, axis=-1)
    X_valid = np.expand_dims(X_valid, axis=-1)
    X_test = np.expand_dims(X_test, axis=-1)
    Y_train = to_categorical(Y_train)
    Y_valid = to_categorical(Y_valid)
    Y_test = to_categorical(Y_test)
    
    input_dim = X_train.shape[1:]
    C = Y_train.shape[1]
    output_dim = Y_train.shape[1]-1

    # loop over model instances (ensembling)
    
    for j in range(num_models):
        start2 = time.time()
        print("\n\n#######################################################")
        print("Split " + str(i) + " Model " + str(j))
        print("#######################################################\n\n")      
    
        mbl = utils.img_model_linear_final(input_dim, output_dim)
        mls = utils.mod_linear_shift(X_tab_train.shape[1])
        model_3d = ontram(mbl, mls)             
        
        model_3d.compile(optimizer=keras.optimizers.Adam(learning_rate=5*1e-5),
                                        loss=ontram_loss(C, batch_size),
                                        metrics=[ontram_acc(C, batch_size)])

        model_name = ("3d_cnn_binary_model_split" + "CIB_LSX" + str(i) + 
                          "_normalized_avg_layer_paper_model_" + last_activation + "_activation_" + str(model_version) + "_" + str(j) + ".h5")

        # Train
        train_data = tf.data.Dataset.from_tensor_slices((X_train, X_tab_train))
        train_labels = tf.data.Dataset.from_tensor_slices((Y_train))
        # Valid
        valid_data = tf.data.Dataset.from_tensor_slices((X_valid, X_tab_valid))
        valid_labels = tf.data.Dataset.from_tensor_slices((Y_valid))
        # Zip
        train_loader = tf.data.Dataset.zip((train_data, train_labels))
        validation_loader = tf.data.Dataset.zip((valid_data, valid_labels))           
        # Data-Sets
        train_dataset = (train_loader.shuffle(len(X_train))
                    .map(train_preprocessing)
                    .batch(batch_size, drop_remainder=True))

        validation_dataset = (validation_loader.batch(batch_size, drop_remainder = True))    
                
        checkpoint_cb = keras.callbacks.ModelCheckpoint(
            filepath = OUTPUT_DIR + model_name,
            verbose = (1 if i == 0 and j == 0 else 0),
            save_weights_only = True,
            monitor = "val_loss", #'val_acc',
            mode = 'min',
            save_best_only = True)

        early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_loss", patience=150, restore_best_weights=True)
        
        
        # Train the model, doing validation at the end of each epoch
        if train:
            hist = model_3d.fit(
            train_dataset,
            validation_data=validation_dataset,
            epochs=250,
            shuffle=True,
            verbose=(1 if i == 0 and j == 0 else 0), 
            callbacks=[checkpoint_cb, early_stopping_cb])

            pkl.dump(hist.history, open(OUTPUT_DIR + "hist_" + model_name[:-2] + "pkl", "wb"), protocol=4)            
        
        end2 = time.time()
        print(" ")   
        print("Duration of Training: " + str(end2-start2))  
        
    end1 = time.time()
    print(" ")   
    print("Duration of Split: " + str(end1-start1))  
        
end0 = time.time()
print(" ")
print("Duration of Everything: " + str(end0-start0))  





################################################################################
Split 0
################################################################################






#######################################################
Split 0 Model 0
#######################################################


Epoch 1/250
54/54 [==============================] - ETA: 0s - loss: 0.6295 - acc: 0.6790
Epoch 00001: val_loss improved from inf to 0.44285, saving model to /tf/notebooks/schnemau/xAI_stroke_3d/weights/10Fold_CIBLSX/3d_cnn_binary_model_splitCIB_LSX0_normalized_avg_layer_paper_model_linear_activation_1_0.h5
54/54 [==============================] - 42s 773ms/step - loss: 0.6295 - acc: 0.6790 - val_loss: 0.4429 - val_acc: 0.8056
Epoch 2/250
54/54 [==============================] - ETA: 0s - loss: 0.5280 - acc: 0.7809
Epoch 00002: val_loss did not improve from 0.44285
54/54 [==============================] - 42s 778ms/step - loss: 0.5280 - acc: 0.7809 - val_loss: 0.4460 - val_acc: 0.8